In [92]:
import bokeh.plotting as bk
from bokeh.layouts import Row, WidgetBox, Column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Paragraph, TextInput, Select, RadioGroup
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

bk.output_notebook()

Loading BokehJS ...

In [90]:
class FilterDesignTool():
    
    def __init__(self,
                 width=800,
                 height=600):
        
        self.filter_figure = bk.figure(width=width, height=height)

        self.response_type = RadioGroup(labels=["Lowpass",
                                        "Highpass",
                                        "Bandpass",
                                        "BandStop",
                                        "Differentiator"],
                                        active=0)
        
        self.update_response_type()
        self.update_design_method()
        self.update_filter_order()
        self.update_options()
        self.update_frequency_specifications()
        self.update_magnitude_specifications()
        
        filter_information_widgets = WidgetBox(Paragraph(text="Filter Information"))
        response_type_widgets = WidgetBox(Paragraph(text="Response Type"),
                                          self.response_type)
        design_method_widgets = WidgetBox(Paragraph(text="Design Method"),
                                          self.design_method, self.design_method_select)
        filter_order_widgets = WidgetBox(Paragraph(text="Filter Order"),
                                         self.specify_order,
                                         self.specify_order_input)
        options_widgets = WidgetBox(Paragraph(text="Options"),
                                    self.density_factor)
        frequency_specifications_widgets = WidgetBox(Paragraph(text="Frequency Specifications"),
                                                     self.freq_units, self.fs, self.fpass, self.fstop)
        magnitude_specifications_widgets = WidgetBox(Paragraph(text="Magnitude Specifications"),
                                                     self.mag_units, self.amp_pass, self.amp_stop)
        
        upper_row = Row(filter_information_widgets,
                        self.filter_figure)
        lower_row = Row(Column(response_type_widgets, design_method_widgets),
                        Column(filter_order_widgets, options_widgets),
                        frequency_specifications_widgets,
                        magnitude_specifications_widgets)
        
        self.layout = Column(upper_row, lower_row)
        
    def update_response_type(self):
        self.response_type = RadioGroup(labels=["Lowpass",
                                                "Highpass",
                                                "Bandpass",
                                                "BandStop",
                                                "Differentiator"],
                                        active=0)
    
    def update_design_method(self):
        self.design_method = RadioGroup(labels=["IIR", "FIR"], active=0)
        self.design_method_select = Select(options=["Butterworth"],
                                           value="Butterworth",
                                           title="Method:")
    
    def update_filter_order(self):
        self.specify_order = RadioGroup(labels=["Min Order", "Specify Order"], active=0)
        self.specify_order_input = TextInput(value='10')
    
    def update_options(self):
        self.density_factor = TextInput(value="20", title="Density Factor")
    
    def update_frequency_specifications(self):
        self.freq_units = Select(value='Hz', title='Units:', options=['Hz', 'Normalised (0-1)'])
        self.fs = TextInput(value='1000.0', title='Fs:')
        self.fpass = TextInput(value='100.0', title='Fpass:')
        self.fstop = TextInput(value='250.0', title='Fstop:')
    
    def update_magnitude_specifications(self):
        self.mag_units = Select(value='dB', title='Magnitude Units:', options=['dB'])
        self.amp_pass = TextInput(value='1.0', title='Apass:')
        self.amp_stop = TextInput(value='80.0', title='Astop:')
        
    def update(self, old, new, attr):
        pass
    
    def modify_doc(self, doc):
        doc.add_root(self.layout)
    

In [91]:
p = FilterDesignTool()
handler = FunctionHandler(p.modify_doc)
app = Application(handler)

bk.show(app)